In [ ]:
!pip install -r requirements.txt

In [47]:
import pathlib
import pickle
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [39]:
documents = []

splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=2500, chunk_overlap=100)

docs_path = pathlib.Path("./mx-docs/docs/")
file_paths = list(docs_path.glob("**/*.md")) + list(docs_path.glob("**/*.mdx"))

for file_path in file_paths:
  with open(file_path, "r") as f:
    content = f.read()

  relative_path = file_path.relative_to(docs_path).with_suffix("")
  metadata = {"source": f"https://docs.multiversx.com/{relative_path}"}

  for chunk in splitter.split_text(content):
    documents.append(Document(page_content=chunk, metadata=metadata))

index = FAISS.from_documents(documents, HuggingFaceEmbeddings())

with open("index.pickle", "wb") as f:
    pickle.dump(index, f)

In [48]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0, max_tokens=-1))

with open("index.pickle", "rb") as f:
  search_index = pickle.load(f)

def query(question):
  selected_docs = search_index.similarity_search(question, k=1)

  response = chain({
    "input_documents": selected_docs,
    "question": f"{question} — Give as much details as possible."
  }, return_only_outputs=True)["output_text"]

  return response

In [49]:
print(query("How to create an ESDT token?"))

 To create an ESDT token, you need to open the MultiversX web wallet, go to the ISSUE section, click on Tokens, enter the required details, review and sign the transaction, and wait for it to be processed. You can find the token identifier on the Explorer page of the issue transaction or from the Web Wallet. To transfer a token, open the MultiversX web wallet, navigate to the Tokens tab, click on Send for the token you want to transfer, introduce the recipient and the amount you want to send, and press Send.
SOURCES: https://docs.multiversx.com/wallet/wallet-token
